<a href="https://colab.research.google.com/github/itsmepriyabrata/priyabrata_ai_python/blob/main/Deep%20Learning%20Algorithm%20part%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep belief networks

In [ ]:
import numpy as np
from collections import namedtuple

# Define RBM class
RBM = namedtuple("RBM", ["weights", "v_bias", "h_bias"])

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def train_rbm(v_visible, learning_rate, epochs):
  """Trains a single RBM layer"""
  # Initialize weights and biases with random values
  num_visible = v_visible.shape[1]
  num_hidden = ... # Define number of hidden units
  weights = np.random.randn(num_visible, num_hidden)
  v_bias = np.zeros(num_visible)
  h_bias = np.zeros(num_hidden)

  for epoch in range(epochs):

    return RBM(weights, v_bias, h_bias)



Transformer networks

In [ ]:
import torch
import torch.nn as nn

class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward):
    super().__init__()
    self.self_attn = nn.MultiheadAttention(d_model, nhead)
    self.linear1 = nn.Linear(d_model, dim_feedforward)
    self.linear2 = nn.Linear(dim_feedforward, d_model)
    self.dropout = nn.Dropout(p=0.1)
    self.layer_norm1 = nn.LayerNorm(d_model)
    self.layer_norm2 = nn.LayerNorm(d_model)

  def forward(self, src, src_mask):
    src = self.self_attn(src, src, src, attn_mask=src_mask)[0]
    src = self.dropout(src)
    src = self.layer_norm1(src + src)

    src = self.linear2(self.dropout(self.linear1(src)))
    src = self.layer_norm2(src + src)
    return src

class TransformerEncoder(nn.Module):
  def __init__(self, d_model, nhead, num_layers, dim_feedforward):
    super().__init__()
    self.layers = nn.ModuleList([TransformerEncoderLayer(d_model, nhead, dim_feedforward) for _ in range(num_layers)])

  def forward(self, src, src_mask):
    for layer in self.layers:
      src = layer(src, src_mask)
    return src

# Example usage (assuming preprocessed data)
src = torch.rand(10, 5, 512)  # Batch size, sequence length, embedding size
src_mask = torch.ones(10, 5, 5)  # Attention mask to prevent padding tokens from influencing attention

transformer_encoder = TransformerEncoder(512, 8, 2, 2048)
encoded_src = transformer_encoder(src, src_mask)

print(encoded_src.shape)


U-net

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

def unet_model(input_size=(128, 128, 1)):
    inputs = layers.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoder
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv5))
    merge6 = layers.concatenate([conv4, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = layers.Conv2D(2, 3, activation='relu', padding='same')(conv9)

    conv10 = layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = models.Model(inputs=inputs, outputs=conv10)

    return model

# Create the U-Net model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Dummy data for demonstration
X_train = np.random.rand(100, 128, 128, 1)
Y_train = np.random.rand(100, 128, 128, 1)
X_test = np.random.rand(20, 128, 128, 1)
Y_test = np.random.rand(20, 128, 128, 1)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0
Y_train = Y_train / 255.0
Y_test = Y_test / 255.0

# Data augmentation
data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(Y_train, augment=True, seed=seed)

image_generator = image_datagen.flow(X_train, batch_size=32, seed=seed)
mask_generator = mask_datagen.flow(Y_train, batch_size=32, seed=seed)

train_generator = zip(image_generator, mask_generator)

# Train the model
model.fit(train_generator, steps_per_epoch=len(X_train) // 32, epochs=50)

# Evaluate the model
model.evaluate(X_test, Y_test)

# Make predictions
predictions = model.predict(X_test)


ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

def resnet_block(input_tensor, filters, kernel_size=3, stride=1, conv_shortcut=True):
    """A block that has a conv layer at shortcut."""
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride)(input_tensor)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = input_tensor

    x = layers.Conv2D(filters, kernel_size, padding='same', strides=stride)(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    return x

def resnet_model(input_shape=(224, 224, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 64, conv_shortcut=False)

    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128, conv_shortcut=False)

    x = resnet_block(x, 256, stride=2)
    x = resnet_block(x, 256, conv_shortcut=False)

    x = resnet_block(x, 512, stride=2)
    x = resnet_block(x, 512, conv_shortcut=False)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

# Create the ResNet model
model = resnet_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Dummy data for demonstration
X_train = np.random.rand(100, 224, 224, 3)
Y_train = np.random.randint(0, 10, size=(100,))
Y_train = tf.keras.utils.to_categorical(Y_train, 10)

X_test = np.random.rand(20, 224, 224, 3)
Y_test = np.random.randint(0, 10, size=(20,))
Y_test = tf.keras.utils.to_categorical(Y_test, 10)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Data augmentation
data_gen_args = dict(rotation_range=20,
                     width_shift_range=0.2,
                     height_shift_range=0.2,
                     shear_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)

# Create data generators
train_generator = image_datagen.flow(X_train, Y_train, batch_size=32)
test_generator = image_datagen.flow(X_test, Y_test, batch_size=32)

# Train the model
model.fit(train_generator, steps_per_epoch=len(X_train) // 32, epochs=10)

# Evaluate the model
model.evaluate(test_generator)

# Make predictions
predictions = model.predict(X_test)


VGGNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
def vgg_model(input_shape=(32, 32, 3), num_classes=10):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 5
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
# Create the VGG model
model = vgg_model()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')
predictions = model.predict(X_test)


170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/50
782/782 [==============================] - 6148s 8s/step - loss: 2.3030 - accuracy: 0.0986 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/50
138/782 [====>.........................] - ETA: 1:20:57 - loss: 2.3027 - accuracy: 0.0986